# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
# CrystalyzeIt: Protein Solubility and Crystallization Advisor

import math
import pandas as pd

# ------------------ Sequence Analysis ------------------ #
def analyze_sequence(seq):
    seq = seq.upper()
    aa_count = {aa: seq.count(aa) for aa in "ACDEFGHIKLMNPQRSTVWY"}
    total = len(seq)
    hydrophobic = sum(aa_count[aa] for aa in "AILMVFWY")
    polar = sum(aa_count[aa] for aa in "STNQ")
    pos = sum(aa_count[aa] for aa in "KRH")
    neg = sum(aa_count[aa] for aa in "DE")
    aromatics = sum(aa_count[aa] for aa in "FWY")
    gp = sum(aa_count[aa] for aa in "GP")
    charge_balance = pos - neg
    hydrophobicity = round((hydrophobic / total) * 100, 2)

    return {
        "Length": total,
        "Hydrophobic": hydrophobic,
        "Polar": polar,
        "Positive": pos,
        "Negative": neg,
        "Aromatic": aromatics,
        "Glycine + Proline (flexibility disruptors)": gp,
        "Hydrophobicity (%)": hydrophobicity,
        "Charge Balance": charge_balance
    }

def print_features(features):
    for k, v in features.items():
        print(f"{k}: {v}")

# ------------------ Solubility Rules ------------------ #
def predict_solubility(protein):
    score = 0
    if protein['length'] < 400:
        score += 1
    if protein['has_tag']:
        score += 2
    if 6.0 < protein['pI'] < 8.5:
        score += 1
    if "e. coli" in protein['expression_system'].lower():
        score += 1
    return score

# ------------------ Screen Prediction ------------------ #
def recommend_screens(pI, cofactor=None, protein_class=None):
    df = pd.read_csv("screen_recommendation_knowledgebase.csv")
    cofactor = cofactor.upper() if cofactor else ""
    protein_class = protein_class or ""

    matches = df[(df["Cofactor"].str.contains(cofactor, na=False)) |
                 (df["Protein_Class"].str.contains(protein_class, na=False))]

    if matches.empty:
        return ["Index", "Morpheus", "PACT Premier", "Crystal Screen HT"]

    top_screens = matches.groupby("Screen")["Score"].max().sort_values(ascending=False)
    return list(top_screens.head(5).index)

# ------------------ Crystallization Conditions ------------------ #
def suggest_crystal_conditions(pI):
    if pI < 6.0:
        return ["Buffer: MES pH 5.5", "Salt: 200 mM NaCl", "Precipitant: PEG 4000"]
    elif 6.0 <= pI <= 8.5:
        return ["Buffer: HEPES pH 7.5", "Salt: 150 mM NaCl", "Precipitant: PEG 3350"]
    else:
        return ["Buffer: CHES pH 9.0", "Salt: 100 mM Ammonium sulfate", "Precipitant: PEG 8000"]

# ------------------ Risk Assessment ------------------ #
def assess_risk(features, solubility_score):
    risk_level = "Low"
    warnings = []
    suggestions = []

    gly_pro_ratio = features["Glycine + Proline (flexibility disruptors)"] / features["Length"]
    if gly_pro_ratio > 0.2:
        risk_level = "Medium"
        warnings.append("⚠ High Gly/Pro content may indicate intrinsic disorder.")
        suggestions.append("Use additives like glycerol or express at lower temperatures.")

    if features["Charge Balance"] < -3 or features["Charge Balance"] > 3:
        risk_level = "Medium"
        warnings.append("⚠ Highly unbalanced surface charge could lead to aggregation.")
        suggestions.append("Optimize buffer pH closer to pI or test solubility tags.")

    if solubility_score <= 2:
        risk_level = "High"
        warnings.append("⚠ Low solubility score based on protein length, tag, and pI.")
        suggestions.append("Consider adding a solubility tag (e.g., MBP/SUMO) or express in Rosetta/Origami strains.")

    return risk_level, warnings, suggestions

# ------------------ User Input ------------------ #
def get_protein_info():
    print(" Welcome to CrystalyzeIt")
    name = input("Protein name: ")
    length = int(input("Sequence length: "))
    has_tag = input("Solubility tag present (yes/no)? ").strip().lower() == 'yes'
    pI = float(input("Isoelectric point (pI): "))
    expression_system = input("Expression system (e.g. E. coli): ")
    sequence = input("Protein sequence (1-letter code): ")
    cofactor = input("Cofactor (e.g. PLP, NAD, None): ")
    protein_class = input("Protein class (e.g. PLP-dependent enzyme): ")

    return {
        "name": name,
        "length": length,
        "has_tag": has_tag,
        "pI": pI,
        "expression_system": expression_system,
        "sequence": sequence,
        "cofactor": cofactor,
        "protein_class": protein_class
    }

# ------------------ Main Program ------------------ #
def main():
    protein = get_protein_info()

    print("\n Protein Feature Summary:")
    features = analyze_sequence(protein['sequence'])
    print_features(features)

    print("\n Solubility Prediction:")
    solubility_score = predict_solubility(protein)
    print(f"Solubility Score: {solubility_score}/5 - {'Good' if solubility_score >= 3 else 'Poor'}")

    print("\n🔮 Recommended Initial Crystallization Screens:")
    screens = recommend_screens(protein['pI'], protein['cofactor'], protein['protein_class'])
    for screen in screens:
        print(f"- {screen}")

    print("\n Suggested Crystallization Conditions:")
    conditions = suggest_crystal_conditions(protein['pI'])
    for c in conditions:
        print(f"- {c}")

    print("\n Risk Assessment:")
    risk_level, warnings, suggestions = assess_risk(features, solubility_score)
    print(f"Risk Level: {risk_level}")
    for warning in warnings:
        print(warning)
    if suggestions:
        print(" Suggestions:")
        for tip in suggestions:
            print(f"- {tip}")

if __name__ == "__main__":
    main()

